In [8]:
# Task1: Establish a connection

from mysql.connector.pooling import MySQLConnectionPool, Error

dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": "SleepWell888"
}

try:
    pool_a = MySQLConnectionPool(
        pool_name = "pool_a",
        pool_size = 2, #default is 5
        **dbconfig
    )

    # Get the connection from the connection pool "pool"
    print("Getting a connection from the pool.")
    connection = pool_a.get_connection()
    
    if connection.is_connected():
        print("Connection is established.")

        # Create a cursor object to interact with the database
        cursor = connection.cursor()
    
    else:
        print("Connection is failed.")

except Error as e:
    print("Error:", e)


Getting a connection from the pool.
Connection is established.


In [9]:
#Task2: Implement a stored procedure called PeakHours
import mysql.connector as connector

try:
    # Establish a database connection
    connection = connector.connect(
        user="root",
        password="SleepWell888",
        database="little_lemon_db"
    )

    if connection.is_connected():
        print("Connection is established.")

        # Step_1: Create the PeakHours stored procedure
        cursor = connection.cursor()

        # Drop the existing stored procedure if it exists
        drop_procedure_query = "DROP PROCEDURE IF EXISTS PeakHours"
        cursor.execute(drop_procedure_query)

        # Create PeakHours stored procedure
        create_procedure_query = """
        CREATE PROCEDURE PeakHours()
        BEGIN
            SELECT HOUR(BookingSlot) AS Hour,
                COUNT(*) AS NumberOfBookings
            FROM Bookings
            GROUP BY HOUR(BookingSlot)
            ORDER BY NumberOfBookings DESC;
        END;
        """


        # Step_2: Run the PeakHours stored procedure
        cursor.execute(create_procedure_query)
        print("Stored procedure PeakHours created.")
        print()

        # Step_3: Invoke callproc to call the stored procedure
        cursor.callproc("PeakHours")

        # Step_4: Fetch the results in a variable called dataset
        results = next(cursor.stored_results())
        dataset = results.fetchall()

        # Step_5: Extract column names
        for column_id in cursor.stored_results():
            columns = [desc[0] for desc in column_id.description]

        # Step_6: Print column names
        print(columns)

        # Step_7: Print sorted data using a loop
        for data in dataset:
            print(data)

except connector.Error as e:
    print("Error:", e)


Connection is established.
Stored procedure PeakHours created.

['Hour', 'NumberOfBookings']
(19, 2)
(15, 1)
(17, 1)
(18, 1)
(20, 1)


In [10]:
#Task3: Implement a stored procedure GuestStatus

import mysql.connector as connector

try:
    # Establish a database connection
    connection = connector.connect(
        user="root",
        password="SleepWell888",
        database="little_lemon_db"
    )

    if connection.is_connected():
        print("Connection is established.")

        # Step_1: Create the GuestStatus stored procedure
        cursor = connection.cursor()

        # Drop the existing stored procedure if it exists
        drop_procedure_query = "DROP PROCEDURE IF EXISTS GuestStatus"
        cursor.execute(drop_procedure_query)

        # Step_2~4: GuestStatus procedure
        create_procedure_query = """
        CREATE PROCEDURE GuestStatus()
        
        BEGIN
        SELECT CONCAT(guest.GuestFirstName,' ', guest.GuestLastName) AS GuestName,
            CASE
                WHEN emp.Role IN ('Manager', 'Assistant Manager') THEN 'Ready to pay'
                WHEN emp.Role = 'Head Chef' THEN 'Ready to serve'
                WHEN emp.Role = 'Assistant Chef' THEN 'Preparing Order'
                WHEN emp.Role = 'Head Waiter' THEN 'Order served'
                ELSE 'Unknown'
            END AS Status
        FROM Bookings AS guest
        LEFT JOIN Employees AS emp ON guest.EmployeeID = emp.EmployeeID;
        END
        """

        # Step_5: Run & Execute query
        cursor.execute(create_procedure_query)
        print("Stored procedure GuestStatus created.")
        print()

        # Step_6: Invoke callproc to call the stored procedure
        cursor.callproc("GuestStatus")

        # Step_7: Fetch results in a variable called dataset
        results = next(cursor.stored_results())
        dataset = results.fetchall()
        
        # Step_8: Extract column names
        for column_id in cursor.stored_results():
            columns = [ column[0] for column in column_id.description ]

        #columns = cursor.column_names

        # Step_9: Print column names
        print(columns)
        
        # Step_10: Print the sorted data
        for row in dataset:
            print(row)
        
        print()

except connector.Error as e:
    print("Error:", e)

finally:
    # Step_11: Close connection
    if connection.is_connected():
        cursor.close()
        print("The cursor is closed.")
        connection.close()
        print("Connection is closed and returned to the pool.")


Connection is established.
Stored procedure GuestStatus created.

['GuestName', 'Status']
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')

The cursor is closed.
Connection is closed and returned to the pool.
